# Program to plot reflectivity from model output
## Calculates reflectivity using Marshall-Palmer 
## Plot on the same domain as the radar images

In [1]:
%matplotlib inline
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import cartopy
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature

# ignora avisos
import warnings
warnings.filterwarnings("ignore")

In [2]:
def plot_SP_map(grid_spc=5, extent=[-48.6, -45.6, -24.65, -22.55]):

    proj = cartopy.crs.PlateCarree(central_longitude=-47.09)
    trans = cartopy.crs.PlateCarree()
    fig, ax = plt.subplots(figsize=(8, 7), facecolor='w', subplot_kw=dict(projection=proj))
    ax.set_extent(extent, crs=trans)
    grid_spc_lat=0.5
    grid_spc=0.5
    shapename_SP = 'filename.shp'

    resol = '10m'  # use data at this scale
    land = cartopy.feature.NaturalEarthFeature('physical', 'land',
           scale=resol, edgecolor='k', facecolor=cfeature.COLORS['land'])
    stt_prv = cartopy.feature.NaturalEarthFeature(category='cultural', 
              name='admin_1_states_provinces_lines',
              scale='10m',facecolor='none')
    ocean = cartopy.feature.NaturalEarthFeature('physical', 'ocean', \
             scale=resol, edgecolor='none', facecolor=cfeature.COLORS['water'])

    ax.add_feature(land, linewidth=1.0 , facecolor='none', edgecolor='gray', zorder=2)
    ax.add_feature(ocean, linewidth=1.0, facecolor='none', edgecolor='gray', zorder=2)
    ax.add_feature(stt_prv, linewidth=1.0, facecolor='none', edgecolor='gray', alpha=1, zorder=2)
    ax.add_geometries(shpreader.Reader(shapename_SP).geometries(), trans,
                     linewidth=1.0, facecolor='none', edgecolor='gray', zorder=2)
    gl = ax.gridlines(crs=trans, xlocs=np.arange(-180, 181, grid_spc),
                      ylocs=np.arange(-80, 90, grid_spc_lat), draw_labels=True)
 
#    gl.xlabels_top = gl.ylabels_right = False
    gl.right_labels = gl.top_labels = False
    gl.xlines = False
    gl.ylines = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    return fig, ax, trans

In [5]:
def plot_refletividade(data, **kwargs):
    fig, ax, trans = plot_SP_map()
    clevs_rain = np.arange(5,65,1)
    color_map = matplotlib.colormaps.get_cmap('NWSRef')
    color_map.set_under('white')
#    cmap = 'NWSRef'
#    cmap.set_under('white')
    ticks=[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65]
    rain_cont = ax.pcolormesh(data['lon'], data['lat'],data, 
                        transform=trans,zorder=1, cmap=color_map, vmin=20, vmax=65) 
    plt.title(kwargs.get('title_plot'), weight='bold', stretch='condensed',
              size='medium', position=(0.55, 1))
    cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
    fig.colorbar(rain_cont, spacing='uniform', ticks=ticks,  extendrect=True,
              label='Reflectivity (dBZ)', orientation='vertical',cax=cax)
    plt.savefig((kwargs.get('title_figure') +
                '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [3]:
dataFile = 'filename.nc'
ds = xr.open_dataset(dataFile)

In [29]:
for t in range (480,553): # Between 16 and 22 UTC, every 5 minutes
    subds = ds.isel(time=t)
    Z = 10*np.log10(200*(subds['pcprate']**1.6))
    date_full = subds['time'].values
    date = np.datetime_as_string(date_full, unit='s').partition('T')[0]
    time = np.datetime_as_string(date_full, unit='s').partition('T')[2]
    time = time.replace(":", "")[:-2]
    title_plot = (date + ' ' + time + ' UTC')  # pt-br
    title_figure = ('Refletividade' + date.replace('-', '') + time)
    print('--- Plotting reflectivity, t = ' + str(t) + ' ---')
    plot_refletividade(Z,title_plot=title_plot,title_figure=title_figure)
    

--- Plotting reflectivityP, t = 480 ---
--- Plotting reflectivityP, t = 481 ---
--- Plotting reflectivityP, t = 482 ---
--- Plotting reflectivityP, t = 483 ---
--- Plotting reflectivityP, t = 484 ---
--- Plotting reflectivityP, t = 485 ---
--- Plotting reflectivityP, t = 486 ---
--- Plotting reflectivityP, t = 487 ---
--- Plotting reflectivityP, t = 488 ---
--- Plotting reflectivityP, t = 489 ---
--- Plotting reflectivityP, t = 490 ---
--- Plotting reflectivityP, t = 491 ---
--- Plotting reflectivityP, t = 492 ---
--- Plotting reflectivityP, t = 493 ---
--- Plotting reflectivityP, t = 494 ---
--- Plotting reflectivityP, t = 495 ---
--- Plotting reflectivityP, t = 496 ---
--- Plotting reflectivityP, t = 497 ---
--- Plotting reflectivityP, t = 498 ---
--- Plotting reflectivityP, t = 499 ---
--- Plotting reflectivityP, t = 500 ---
--- Plotting reflectivityP, t = 501 ---
--- Plotting reflectivityP, t = 502 ---
--- Plotting reflectivityP, t = 503 ---
--- Plotting reflectivityP, t = 504 ---


In [7]:
subds = ds.isel(time=509)
Z = 10*np.log10(200*(subds['pcprate']**1.6))
date_full = subds['time'].values
date = np.datetime_as_string(date_full, unit='s').partition('T')[0]
time = np.datetime_as_string(date_full, unit='s').partition('T')[2]
time = time.replace(":", "")[:-2]
title_plot = (date + ' ' + time + ' UTC')  # pt-br
title_figure = ('Reflectivity' + date.replace('-', '') + time)
plot_refletividade(Z,title_plot=title_plot,title_figure=title_figure)